# Copy Paste Other Functions - b/c import wasn't working

In [1]:
def concat_DfColumn(df,columnIndicator):
    import pandas
    tempSeries = df[columnIndicator]
    strcat = tempSeries.str.cat(sep=', ')
    return strcat

In [2]:
def get_KeyPhrases(textInput, k = 15, version = 'Summary'):
    #version = Summary or version = PMI
    import nltk
    nltk.download('punkt',quiet='true')
    nltk.download("averaged_perceptron_tagger",quiet='true')
    nltk.download("wordnet",quiet='true')
    nltk.download("brown",quiet='true')
    from nltk.data import find
    from nltk.tag import PerceptronTagger
   

    #setting up tagger
    #(from http://stackoverflow.com/a/35964709)
    PICKLE = "averaged_perceptron_tagger.pickle"
    AP_MODEL_LOC = 'file:'+str(find('taggers/averaged_perceptron_tagger/'+PICKLE))
    tagger = PerceptronTagger(load=False)
    tagger.load(AP_MODEL_LOC)
    pos_tag = tagger.tag

    lemmatizer = nltk.WordNetLemmatizer()
    stemmer = nltk.stem.porter.PorterStemmer()
    
    
    # This grammar is described in the paper by S. N. Kim,
    # T. Baldwin, and M.-Y. Kan.
    # Evaluating n-gram based evaluation metrics for automatic
    # keyphrase extraction.
    # Technical report, University of Melbourne, Melbourne 2010.
    from nltk.corpus import stopwords
    stopwords = stopwords.words('english')


    def leaves(tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()

    def acceptable_word(word):
        """Checks conditions for acceptable word: length, stopword."""
        accepted = bool(2 < len(word) and word.lower() not in stopwords)
        return accepted        

    def normalise(word):
        """Normalises words to lowercase and stems and lemmatizes it."""
        word = word.lower()
        word = stemmer.stem(word)
        word = lemmatizer.lemmatize(word)
        return word

    def get_terms(tree):
        for leaf in leaves(tree):
            #can modify normalise to w.lower() if dont want to normalize word
            term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
            yield term
        
    def get_nounPhrases(textInput, minWordLength = 2):
        lemmatizer = nltk.WordNetLemmatizer()
        stemmer = nltk.stem.porter.PorterStemmer()

        grammar = r"""

        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
                  """

        chunker = nltk.RegexpParser(grammar)
    
        toks = nltk.word_tokenize(textInput)
        #print(toks)
        pos_tag = tagger.tag
        postoks = pos_tag(toks)

        tree = chunker.parse(postoks)
        terms = get_terms(tree)
   
        nounPhraseList = []
        for tid,term in enumerate(terms):
            templist = []
            for wid, word in enumerate(term):
                #print("TID: ",tid," WID: ",(wid+1), word)
                templist.append(word)
        
            s = " "
            nounPhraseList.append(s.join(templist))

        nounPhraseList = [word for word in nounPhraseList if len(word.split())>=minWordLength]
        return nounPhraseList
    
    import pandas
    from collections import Counter
    counter = Counter()
    for nounPhrase in  get_nounPhrases(textInput):
        #print(nounPhrase)
        counter.update([nounPhrase])
    if version.lower() == 'summary':       
        topkNPdf =pandas.DataFrame([[key,value] for key,value in counter.items()],columns=['Term','Frequency'])
        #topkNPdf = topkNPdf.reset_index(drop=True)

        #if less than max (15), use correct number of key phrases
        if topkNPdf.shape[0]<k:
            print("\n \nTop" ,topkNPdf.shape[0], "key phrases (minimum phrase length = 2 ): \n")
        else:
            print("\n \nTop" ,k, "key phrases (minimum phrase length = 2): \n") 


        topkNPdf= topkNPdf.sort_values('Frequency', axis=0, ascending=False).head(k)
        topkNPdf = topkNPdf.reset_index(drop=True)
        return topkNPdf
    
    elif version.lower() == 'pmi':
        return counter.most_common(k);




    
    
    
    

# PMI Function

In [3]:
def cleanData(df):
    import re
    URL_REGEX_1 = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    SHOPIFY_REGEX = r"[S|s]hopify"
    HIRING_REGEX = r"[H|h]iring"
    POPULARCOMMENT_REGEX = r"\**Most Popular Comments\**"
    HISTORY_REGEX = r"[R|r]ecent [S|s]ubmission [H|h]istory "
    
    dfCopy = df.copy()
    dfCopy['bodyCopy'] =dfCopy['body']
    
    #list to store ids of comments to keep 
    commentKeep =[]

    for cid,comment in enumerate(dfCopy['bodyCopy']):
        #print(cid)
        #print(dfCopy.loc[dfCopy['body']==comment].index.values[0])
        if (len(re.findall(URL_REGEX_1,comment))>0):
            dfCopy['bodyCopy'][cid] = re.sub(URL_REGEX_1,'WEBSITE_FILLER',comment)
            #print(dfCopy['bodyCopy'][cid])

    # 2nd loop to check if post meets requirements after subbing in URL FILLER    
    for cid,comment in enumerate(dfCopy['bodyCopy']):    
        #only keep comments with links if they mention shopify w/o it occuring in link    
        if (len(re.findall(SHOPIFY_REGEX,comment))>0):
                #remove any post with 'Hiring' or 'Recent Submission History'
                if(len(re.findall(HIRING_REGEX,comment))==0 and  len(re.findall(HISTORY_REGEX,comment))==0 and len(re.findall(POPULARCOMMENT_REGEX,comment))==0):
                    commentKeep.append(cid)


    print("****Done****")
    dfClean=df.loc[commentKeep]
    
    
    return dfClean

In [4]:
# Calcualte MI scores
def miCalc(df, topK, scoreMetric = 'vader'):
    import sklearn
    import sklearn.metrics as metrics
   
    gtScore = []
    threshold = 0
    
    # Convet score metric to binary values
    for i in range(len(df)):
        if df[scoreMetric].as_matrix()[i]>threshold:
            gtScore.append(1)
        else:
            gtScore.append(0)            

    # Perform MI Analysis for each word
    miScore = []
    for word, wordCount in topK:
        miScore.append([word]+[metrics.mutual_info_score(gtScore, df[word].as_matrix())])
       
    
    miScoredf = pandas.DataFrame(miScore).sort_values(1,ascending=0)
    miScoredf = miScoredf.reset_index(drop = True)
    miScoredf.columns = ['Word','MI Score']
    return miScoredf

In [5]:
def calcPMI(df, MItype = 'mi'):
    import pandas
    import numpy as np
    import math
    import nltk
    from nltk.sentiment import SentimentAnalyzer
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    #from nltk.sentiment.util import *
    from nltk import tokenize
    from nltk.corpus import stopwords
   
    df =cleanData(df)
    
    def commentPolarity(df):
        commentContent = df['body'].as_matrix()
        #Instantiation
        sid = SentimentIntensityAnalyzer()
        pdlist = []
        ##Assign Vader score to individual review using Vader compound score
        ##creating a list of the reviews along with their polarity score as assigned by Vader
        ##list containing a list where each element is [review,polarity]
        for rownum, comment in enumerate(commentContent):
            ss = sid.polarity_scores(comment)
            pdlist.append([comment,ss['compound']])
            if (rownum % 100 == 1):
                    print("processed %d reviews" % (rownum+1))
        print("****Done****")
        commentDf = pandas.DataFrame(pdlist)
        commentDf.columns = ['commentCol','vader']
#         commentDf.head()
        return commentDf
        
    def commentTermDF(df):
        
        top_k =  get_KeyPhrases(concat_DfColumn(df,'commentCol'), k = 100, version = 'pmi')

        freqReview = []
        #create a list, one entry for each review; the entry is binary list indicating whether this review as the term i or not in the top k
        for i in range(len(df)):
            commentKP = get_KeyPhrases(df['commentCol'][i], k = 100, version = 'pmi')
        #     print('commentKP: ' , commentKP)
            keyPhraseDict = {commentKP[i][0]: commentKP[i][1] for i in range(0, len(commentKP))}
        #     print('\n keyPhraseDict',keyPhraseDict)
            topkinComment = [1 if  keyPhraseDict.get(word, 0) > 0 else 0 for (word,wordCount) in top_k]
        #     print('\n topkinComment',topkinComment)
            freqReview.append(topkinComment)

        freqReviewDf = pandas.DataFrame(freqReview)
        dfName = []
        for c in top_k:
            dfName.append(c[0])
            #print(c)
        freqReviewDf.columns = dfName
#         freqReviewDf.head()
        return top_k,freqReviewDf
    
    
    df1 = commentPolarity(df)
    top_k, df2 = commentTermDF(df1)
    finalcommentDf = df1.join(df2)
    df_reindexed = df.reset_index(drop=True)
    finaldf = df_reindexed.join(finalcommentDf)
    if MItype.lower() == 'mi':
        miDF = miCalc(finaldf, top_k, scoreMetric = 'vader')
        return miDF
    
    

# Example

In [6]:
#!pip --quiet install nltk
import nltk
nltk.download("vader_lexicon")
nltk.download("stopwords")

#for candidate key phrase code
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download("wordnet")
nltk.download("brown")

from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.tag import PerceptronTagger
from nltk.data import find
import pandas
import numpy as np
import math


import json
from collections import Counter

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!

C:\Users\Jason\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [7]:
#location of data
DATA_DIR = r'C:\Users\Jason\MIE490 - Capstone - Shopify\Data\Comments'

In [8]:
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
import os
file_in = []
# os.walk - 
#Generate the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory
#in the tree rooted at directory top (including top itself), it yields a 3-tuple (dirpath, dirnames, filenames).
for root, dirs, files in os.walk(DATA_DIR):
    #print(root  + '\n')
    #print(files)
    #print(files +"\n")
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    #print(filePaths)
    file_in.extend(filePaths)
    #print(file_in)

In [9]:
dataDf = pandas.DataFrame()
#following code courtesy of: https://www.reddit.com/r/MachineLearning/comments/33eglq/python_help_jsoncsv_pandas/
for file in file_in:
    with open(file, 'r') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"
    # now, load it into pandas
    dataDf = dataDf.append(pandas.read_json(data_json_str),ignore_index=True)
    #print(dataDf.head())


In [10]:
dataDf.head(n=2)

,archived,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,downs,edited,...,link_id,name,parent_id,removal_reason,retrieved_on,score,score_hidden,subreddit,subreddit_id,ups
0,False,battery_bot,None,None,\n&amp;nbsp; \n\n ^Brand | ^Model | ^(Rated C...,0,1427847662,None,0,0,...,t3_30zopt,t1_cpxa6po,t1_cpxa6mw,NaN,1432126807,1,False,electronic_cigarette,t5_2qmlu,1
1,False,uncle_dubya,,the funky uncle.,"yeah, that's where i saw them. i thought about...",0,1427849075,None,0,0,...,t3_30x0l7,t1_cpxb06p,t1_cpx0fy0,NaN,1432127192,1,False,Wet_Shavers,t5_32kax,1


In [11]:
dataDf.shape

(3870, 22)

In [12]:
cleanDF = cleanData(dataDf)

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


****Done****


In [13]:
cleanDF.shape

(1720, 22)

In [14]:
cleanDF.columns

Index(['archived', 'author', 'author_flair_css_class', 'author_flair_text',
       'body', 'controversiality', 'created_utc', 'distinguished', 'downs',
       'edited', 'gilded', 'id', 'link_id', 'name', 'parent_id',
       'removal_reason', 'retrieved_on', 'score', 'score_hidden', 'subreddit',
       'subreddit_id', 'ups'],
      dtype='object')

In [15]:
temp = calcPMI(dataDf, MItype = 'mi')

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


****Done****
processed 2 reviews
processed 102 reviews
processed 202 reviews
processed 302 reviews
processed 402 reviews
processed 502 reviews
processed 602 reviews
processed 702 reviews
processed 802 reviews
processed 902 reviews
processed 1002 reviews
processed 1102 reviews
processed 1202 reviews
processed 1302 reviews
processed 1402 reviews
processed 1502 reviews
processed 1602 reviews
processed 1702 reviews
****Done****


In [16]:
temp

,Word,MI Score
0,good luck,5.602753e-03
1,free ship,1.607251e-03
2,littl bit,1.460651e-03
3,whole thing,1.460651e-03
4,front page,1.167746e-03
5,bank account,1.167746e-03
6,much time,1.167746e-03
7,host solut,1.167746e-03
8,free theme,1.167746e-03
9,ecommerc platform,1.161278e-03


In [17]:
temp.shape

(100, 2)